In [1]:
import pandas as pd
import boto3

In [ ]:
import boto3
import pandas as pd
from io import StringIO

# Define your bucket name and file key (file path in S3)
BUCKET_NAME = "blue-blood-data"
FILE_KEY = "BigQuery Blue Blood DB Data.csv"  # Change to your actual file path in S3

# Create an S3 client
s3 = boto3.client("s3")

# Fetch the file from S3
response = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)

# Read the CSV file into a pandas DataFrame
csv_content = response["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(csv_content))

# Print DataFrame
print(df.head())

   subject_id   prescription_start  \
0       10013  2125-10-05T00:00:00   
1       10019  2163-05-15T00:00:00   
2       10027  2190-07-14T00:00:00   
3       10040  2147-02-24T00:00:00   
4       10042  2147-02-07T00:00:00   

                                  prescription_drugs  \
0  Dobutamine, Vasopressin, Vasopressin, Ceftriax...   
1  Sodium Chloride , Sodium Chloride , Norepineph...   
2  Dextrose 50%, Amiodarone HCl, Amiodarone HCl, ...   
3  Magnesium Sulfate, Multivitamins, Bisacodyl, F...   
4  Nitroprusside Sodium, Meperidine, Midazolam HC...   

                         prescription_drug_name_poes  \
0  Azithromycin, Azithromycin, Morphine Sulfate, ...   
1  Sodium Bicarbonate, Artificial Tear Ointment, ...   
2  Dextrose 50%, Amiodarone HCl, HydrALAZINE HCl,...   
3  Magnesium Sulfate, Multivitamins, Bisacodyl, F...   
4  Meperidine, Midazolam HCl, Magnesium Sulfate, ...   

                     prescription_formulary_drug_cds  \
0  DOBPREM, VASO20I, VASO20I, CEFX1F, DOP

In [19]:
# Filter and keep as DataFrame instead of converting to NumPy array
filtered_df = df.filter(regex=r"^(pre|post_)")

# Now you can use min() while preserving column names
min_values = get_min_for_numeric_columns(filtered_df)
print(min_values)

pre_ph                      7.060000
pre_pco2                   17.000000
pre_po2                    51.000000
pre_bicarbonate                  NaN
pre_baseexcess            -16.000000
pre_totalco2               10.000000
pre_hematocrit             26.000000
pre_hemoglobin              8.700000
pre_sodium                127.000000
pre_potassium               2.600000
pre_chloride               92.000000
pre_glucose                77.000000
pre_lactate                 0.700000
pre_so2                    91.000000
pre_spo2                   89.000000
pre_fio2_chartevents       28.000000
pre_aado2_calc             26.890000
pre_pao2fio2               51.000000
pre_temperature            35.000000
pre_fio2                   30.000000
pre_aado2                 226.000000
pre_carboxyhemoglobin       1.000000
pre_methemoglobin                NaN
pre_calcium                 0.700000
pre_intubated                    NaN
post_ph                     7.120000
post_pco2                  12.000000
p

In [23]:
import numpy as np

for col in df.select_dtypes(include=[np.number]).columns:
    if (df[col] < 0).any():
        print(f"Column '{col}' contains negative values.", df[col].values.min())

Column 'pre_baseexcess' contains negative values. -16.0
Column 'post_baseexcess' contains negative values. -17.0


In [24]:
filtered_df.fillna(-200, inplace=True)

/var/folders/ml/sklxcrvd7vjc9kdgrz6rgb9w0000gn/T/ipykernel_57139/2117843922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.fillna(-200, inplace=True)


In [26]:
# find max value of column
# find max value of column and normalize only if column is numeric
for col in filtered_df.columns:
    # Check if the column contains numeric data
    if pd.api.types.is_numeric_dtype(filtered_df[col]):
        max_val = filtered_df[col].max()
        # Only normalize if max_val is not zero to avoid division by zero
        if max_val != 0:
            filtered_df[col] = filtered_df[col] / max_val
# divide each value in column by max value

/var/folders/ml/sklxcrvd7vjc9kdgrz6rgb9w0000gn/T/ipykernel_57139/1689826925.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col] = filtered_df[col] / max_val


In [27]:
filtered_df.head()

,prescription_start,prescription_drugs,prescription_drug_name_poes,prescription_formulary_drug_cds,prescription_gsns,prescription_ndcs,prescription_prod_strengths,prescription_dose_val_rxs,prescription_dose_unit_rxs,prescription_form_val_disps,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,2125-10-05T00:00:00,"Dobutamine, Vasopressin, Vasopressin, Ceftriax...","Azithromycin, Azithromycin, Morphine Sulfate, ...","DOBPREM, VASO20I, VASO20I, CEFX1F, DOPA400PM, ...","21502, 6612, 6612, 9156, 4975, 4975, 6522, 166...","74234632, 54569391600, 54569391600, 4200278, 7...","250MG PREMIX, 20U/ML VIAL, 20U/ML VIAL, 1GM FR...","250, 100, 100, 1, 400, 400, 25000, 100, 250, 5...","mg, UNIT, UNIT, gm, mg, mg, UNIT, mg, mg, mg, ...","1, 5, 5, 1, 1, 1, 1, 1, 1, 2, 0.5, 0.1-0.5, 1,...",...,0.6,0.374676,0.372892,-5.141388,-2.0,-0.315457,1.0,1.0,-165.289256,1.0
1,2163-05-15T00:00:00,"Sodium Chloride , Sodium Chloride , Norepineph...","Sodium Bicarbonate, Artificial Tear Ointment, ...","NACL23.4I, NACL23.4I, LEVO4I, SW3000, SW3000, ...","1220, 1220, 4939, 9825, 9825, 9825, 9825, 9825...","517293025, 517293025, 74144304, 74797308, 7479...","4mEq/ml-30ml, 4mEq/ml-30ml, 4MG/4ML AMP, 3000 ...","231, 231, 8, 3000, 3000, 3000, 3000, 3000, 300...","mEq, mEq, mg, ml, ml, ml, ml, ml, ml, ml, mg, ...","57.75, 57.75, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",...,-2.0,-0.323625,-0.389105,-5.141388,-2.0,-0.315457,1.0,1.0,0.760331,1.0
2,2190-07-14T00:00:00,"Dextrose 50%, Amiodarone HCl, Amiodarone HCl, ...","Dextrose 50%, Amiodarone HCl, HydrALAZINE HCl,...","DEX50SY, AMIO150I, AMIO150I, AMIO150I, HYDZ20I...","1989, 50676, 50676, 50676, 283, 283, 8205, 197...","409490234, 55390005710, 55390005710, 553900057...","50mL Syringe, 150mg/3mL Vial, 150mg/3mL Vial, ...","25, 600, 600, 150, 5, 5, 20, 250, 250, 10","gm, mg, mg, mg, mg, mg, mg, ml, ml, UNIT","1, 4, 4, 1, 0.25, 0.25, 0.5, 1, 1, 0.1",...,0.4,0.183172,0.642023,-5.141388,-2.0,-0.315457,1.0,1.0,-165.289256,1.0
3,2147-02-24T00:00:00,"Magnesium Sulfate, Multivitamins, Bisacodyl, F...","Magnesium Sulfate, Multivitamins, Bisacodyl, F...","MAGS1I, MVI, BISA5, FERR325, FURO40I, PANT40, ...","1417, 2524, 2947, 1645, 8205, 27462, 6549, 300...","517260225, 55499120401, 51079000620, 510790116...","1gm / 2 mL Vial, 1 CAP, 5MG TAB, 325MG TAB, 40...","2, 1, 10, 325, 40, 40, 5000, 100, 40, 1","gm, CAP, mg, mg, mg, mg, UNIT, mg, mEq, TAB","4, 1, 2, 1, 1, 1, 1, 1, 4, 1",...,0.4,0.120874,0.768482,0.974293,0.4,-0.315457,1.0,1.0,-165.289256,1.0
4,2147-02-07T00:00:00,"Nitroprusside Sodium, Meperidine, Midazolam HC...","Meperidine, Midazolam HCl, Magnesium Sulfate, ...","NITP50I, MEPE50I, MIDA2I, MAGS1I, LEVO4I, VANC...","19652, 4051, 3779, 1412, 4939, 9329, 23159, 15...","74302401, 54354563, 74258702, 74407532, 741443...","25mg/ml-2ml, 50 mg Syringe, 2mg/2mL Vial, 1gm/...","50, 12.5, 1-2, 2, 4, 1000, 20, 100, 0.5-4.0, 3...","mg, mg, mg, gm, mg, mg, mg, mg, mg, mg, mg, mg...","2, 0.25, 1-2, 4, 1, 1, 1, 1, 0.05-0.4, 1, 1, 1...",...,-2.0,-0.323625,-0.389105,-5.141388,-2.0,-0.315457,1.0,1.0,0.900826,1.0
